In [1]:
%%time
%%sh

# The name of our algorithm
# Makoto.Sano@Mack-the-Psych.com
algorithm_name=sagemaker-vdok3-memn2n

cd container

chmod +x vdok3_sage/train
chmod +x vdok3_sage/serve

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.
aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1

if [ $? -ne 0 ]
then
    aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly
$(aws ecr get-login --region ${region} --no-include-email)

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build  -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

Login Succeeded
Sending build context to Docker daemon  40.45kB
Step 1/16 : FROM continuumio/anaconda3:4.4.0
4.4.0: Pulling from continuumio/anaconda3
8ad8b3f87b37: Pulling fs layer
fb691515f399: Pulling fs layer
6c3051db0635: Pulling fs layer
66faddd8f0d6: Pulling fs layer
66faddd8f0d6: Waiting
8ad8b3f87b37: Verifying Checksum
8ad8b3f87b37: Download complete
fb691515f399: Verifying Checksum
fb691515f399: Download complete
66faddd8f0d6: Verifying Checksum
66faddd8f0d6: Download complete
8ad8b3f87b37: Pull complete
6c3051db0635: Verifying Checksum
6c3051db0635: Download complete
fb691515f399: Pull complete
6c3051db0635: Pull complete
66faddd8f0d6: Pull complete
Digest: sha256:c6bb52bffe028b4b436b085afa4044db9b3d687a95468c92578467c9c2d4ac31
Status: Downloaded newer image for continuumio/anaconda3:4.4.0
 ---> 795ad88c47ff
Step 2/16 : RUN conda install -c anaconda setuptools
 ---> Running in 0f6b4fa3d1ff
Fetching package metadata ...........
Solving package specifications: .

Package plan 

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



CPU times: user 40.7 ms, sys: 9.21 ms, total: 49.9 ms
Wall time: 8min 18s


In [2]:
# S3 prefix
# Makoto.Sano@Mack-the-Psych.com
prefix = 'vdok3_memn2n_trial'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [3]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [4]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [5]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name

# Makoto.Sano@Mack-the-Psych.com
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-vdok3-memn2n:latest'.format(account, region)
vdok3memn2n = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess)
vdok3memn2n.fit(data_location)

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-09-27 08:11:51 Starting - Starting the training job...
2020-09-27 08:11:53 Starting - Launching requested ML instances.........
2020-09-27 08:13:24 Starting - Preparing the instances for training...
2020-09-27 08:14:06 Downloading - Downloading input data
2020-09-27 08:14:06 Training - Downloading the training image.........
2020-09-27 08:15:35 Training - Training image download completed. Training in progress./opt/conda/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters

Starting the training.
Building model...


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons


In [6]:
from sagemaker.predictor import csv_serializer
predictor = vdok3memn2n.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


-----------------!

In [7]:
df_in = pd.read_csv('data/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv')
np_in = np.vstack((np.array(df_in.columns), df_in.to_numpy()))
print(predictor.predict(np_in).decode('utf-8'))

Score_Class
1
1
2
0



In [8]:
sess.delete_endpoint(predictor.endpoint)

In [9]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = vdok3memn2n.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv')

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
Using already existing model: sagemaker-vdok3-memn2n-2020-09-27-08-11-50-836


In [10]:
transformer.transform(data_location + '/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv', content_type='text/csv', split_type='Line')
transformer.wait()

s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out".format(transform_output_folder), '/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out')
with open('/tmp/Head4-Serialized-Def-ELVA.PILOT.POST-TEST.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

...................................Starting the inference server with 4 workers.
2020/09/27 08:31:18 [crit] 12#0: *1 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [27/Sep/2020:08:31:18 +0000] "GET /ping HTTP/1.1" 502 172 "-" "Go-http-client/1.1"
2020/09/27 08:31:18 [crit] 12#0: *3 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, client: 169.254.255.130, server: , request: "GET /ping HTTP/1.1", upstream: "http://unix:/tmp/gunicorn.sock:/ping", host: "169.254.255.131:8080"
169.254.255.130 - - [27/Sep/2020:08:31:18 +0000] "GET /ping HTTP/1.1" 502 172 "-" "Go-http-client/1.1"
2020/09/27 08:31:18 [crit] 12#0: *5 connect() to unix:/tmp/gunicorn.sock failed (2: No such file or directory) while connecting to upstream, 